# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17", base_url="https://api.xiaoai.plus/v1")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条蜿蜒穿过绿色草地的木质栈道。画面中的草地呈现出丰富的绿色，配以点缀的野花和灌木丛，营造出一种自然而宁静的环境。栈道似乎通往远处的未知目的地，引导观者的视线向前延伸。天空晴朗，蓝天中散布着几朵白云，增添了画面的宁静感和深远感。整体上，这幅图片传递了一种和谐与平静的大自然之美，是户外散步和自然观察的理想场所。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [2]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17", base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [3]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的表现方式，通过图像编辑，将柴犬融合入了肌肉发达的人体形象上。图中分为两部分，左边的肌肉柴犬被称为“16岁的我”，右边则是普通体态的柴犬，代表“工作后的我”。图中左侧的肌肉柴犬配文表达了年轻时的雄心壮志，如“我刚走上社会时身体超级健康，充满野心，人与人之间的关系大部分还蛮简单”，而右侧的普通柴犬配文则表达了工作后的疲惫与无奈，如“好累好困，好想要躺着不动，多做不如少做，做得好不如做的巧，跟着感觉走，人际关系狗血”。

这种对比的幽默表达了许多人对于成长和工作


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [23]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17", base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="我将请你帮我识别图片中的手写文字。图中可能会有简体中文，英文，也可能有手画的小人。请你帮我提取出文字，并总结其中心思想。", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/cxk2.png)

In [18]:
content = query_base64_image_description("./images/cxk2.png")
print(content)

这张图片显示了一个人在运动场或体育馆内进行篮球运动。主角穿着黑色上衣、灰色长裤和黑色运动鞋，看起来正在做运球动作。他的动作显得很有活力，而场地简约的背景则使得整个场景专注于这个篮球运动的动态。整体上，图片体现了一种运动和活力的氛围。


#### 使用 GPT-4V 识别手写体笔记

![](./images/gpt-4v.jpg)

In [26]:
content = query_base64_image_description("images/gpt-4v.jpg")
print(content)

图中的手写文字是：“我最喜欢可口可乐和中央情报局特工C.X.K. 我喜欢篮球、rap、旅行！”

此文字表达了作者的个人喜好，包括喜爱喝可口可乐、对某特工C.X.K的喜爱（可能是虚构的或是内部玩笑），此外还喜欢打篮球、听rap音乐和旅行。

图中还画了一个小人，小人戴着眼镜，看起来很开心，手里拿着篮球，穿着常见的篮球服饰。这可能意味着作者自己或是他喜欢的某个角色。从整体来看，这张图片有趣并充满个性。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [27]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

图中的手写文字是：“我最喜欢可口可乐和中央情报局特工C.X.K. 我喜欢篮球、rap、旅行！”

此文字表达了作者的个人喜好，包括喜爱喝可口可乐、对某特工C.X.K的喜爱（可能是虚构的或是内部玩笑），此外还喜欢打篮球、听rap音乐和旅行。

图中还画了一个小人，小人戴着眼镜，看起来很开心，手里拿着篮球，穿着常见的篮球服饰。这可能意味着作者自己或是他喜欢的某个角色。从整体来看，这张图片有趣并充满个性。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。